In [240]:
# Import the neccesary libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sea

In [241]:
# Note: The file path of the csv files is dependent on the location where the file is saved and as such is subject to chage

# Read in the gpu data
gpu_price_df = pd.read_csv('/Users/sparsh/Documents/CS176_project/pc_data/FACT_GPU_PRICE.csv')
gpu_product_df = pd.read_csv('/Users/sparsh/Documents/CS176_project/pc_data/DIM_GPU_PROD.csv')

# Read in the cpu data
cpu_price_df = pd.read_csv('/Users/sparsh/Documents/CS176_project/pc_data/FACT_CPU_PRICE.csv')
cpu_product_df = pd.read_csv('/Users/sparsh/Documents/CS176_project/pc_data/DIM_CPU_PROD.csv')

# Read in the ram data
ram_price_df = pd.read_csv('/Users/sparsh/Documents/CS176_project/pc_data/FACT_RAM_PRICE.csv')
ram_product_df = pd.read_csv('/Users/sparsh/Documents/CS176_project/pc_data/DIM_RAM_PROD.csv')

# Read in other data (merchant and region)
merchant_df = pd.read_csv('/Users/sparsh/Documents/CS176_project/pc_data/DIM_MERCHANT.csv')
region_df = pd.read_csv('/Users/sparsh/Documents/CS176_project/pc_data/DIM_REGION.csv')

In [242]:
merchant_df

,Id,Merchant
0,1,1stWave Technologies
1,2,Adorama
2,3,Alternate
3,4,Alternate Italia
4,5,Alza
...,...,...
76,77,Storm Computers
77,78,SuperBiiz
78,79,Umart
79,80,Vuugo


In [243]:
region_df

,Id,Code,Currency
0,1,au,AUD
1,2,be,EUR
2,3,ca,CAD
3,4,de,EUR
4,5,es,EUR
5,6,fr,EUR
6,7,ie,EUR
7,8,it,EUR
8,9,nz,NZD
9,10,uk,GBP


In [244]:
# Change the codes to full country name for clarity 
region_df["Country"] = ["Austrailia","Belgium","Canada","Germany","Spain","France","Ireland","Italy","New Zealand","United Kingdom","United States Of AMERICA RAAH🦅🦅🦅"]

# Drop the local currency as only USD will be used for standardized comparision
region_df = region_df.drop(columns=['Currency', 'Code'])

# Change column name for easier merging
region_df = region_df.rename(columns={'Id': 'RegionId'})

In [245]:
# Change the column name for easier merging
merchant_df = merchant_df.rename(columns={'Id': 'MerchantId'})

In [246]:
gpu_price_df

,ProdId,TimeId,RegionId,MerchantId,Price_USD,Price_Original
0,1,20140917,4,32,601.738413,463.900
1,1,20140918,4,32,551.885276,425.870
2,1,20140919,4,32,548.098896,424.530
3,1,20140920,4,32,545.093860,424.530
4,1,20140921,4,32,544.674245,424.530
...,...,...,...,...,...,...
997151,2054,20180312,4,32,261.254103,212.090
997152,2054,20180313,4,32,263.058996,212.090
997153,2054,20180314,4,32,261.517908,211.415
997154,2054,20180315,4,32,260.038547,211.150


In [247]:
gpu_product_df

,Id,Processor_Manufacturer,Processor,GPU_Manufacturer,Memory_Capacity,Memory_Type
0,1,AMD,C420,Matrox,2.000,GDDR5
1,2,AMD,C680,Matrox,2.000,GDDR5
2,3,AMD,C680,Matrox,4.000,GDDR5
3,4,AMD,C900,Matrox,4.000,GDDR5
4,5,AMD,FireGL V3300,AMD,0.125,GDDR2
...,...,...,...,...,...,...
2049,2050,NVidia,Quadro P5000,PNY,16.000,GDDR5X
2050,2051,NVidia,Quadro P600,PNY,2.000,GDDR5
2051,2052,NVidia,Quadro P6000,PNY,24.000,GDDR5
2052,2053,NVidia,Quadro P6000,PNY,24.000,GDDR5X


In [248]:
# Merge the price data frames with the region data frame to have the names of the country instead of the region ID
gpu_price_df = pd.merge(gpu_price_df,region_df,on="RegionId")
cpu_price_df = pd.merge(cpu_price_df,region_df,on="RegionId")
ram_price_df = pd.merge(ram_price_df,region_df,on="RegionId")

# Drop the RegionId column as it is no longer needed since the country names are displayed
gpu_price_df = gpu_price_df.drop(columns=['RegionId'])
cpu_price_df = cpu_price_df.drop(columns=['RegionId'])
ram_price_df = ram_price_df.drop(columns=['RegionId'])

In [249]:
# Merge the price data frames with the merchant data frame to have the names of the merchant instead of the merchant ID
gpu_price_df = pd.merge(gpu_price_df,merchant_df,on="MerchantId")
cpu_price_df = pd.merge(cpu_price_df,merchant_df,on="MerchantId")
ram_price_df = pd.merge(ram_price_df,merchant_df,on="MerchantId")

# Drop the MerchantId column as it is no longer needed since the merchant names are displayed
gpu_price_df = gpu_price_df.drop(columns=['MerchantId'])
cpu_price_df = cpu_price_df.drop(columns=['MerchantId'])
ram_price_df = ram_price_df.drop(columns=['MerchantId'])

In [250]:
# Merge the price and the product dataframe on the Id
merged_gpu = pd.merge(gpu_product_df,gpu_price_df,left_on="Id",right_on="ProdId")

In [251]:
# Drop ProdId as it is a repetitive column, TimeId is not relevant to the project, and Price_Orginial because USD will be use for comparisions not local currencies
merged_gpu = merged_gpu.drop(columns=['ProdId','TimeId','Price_Original'])

In [252]:
# Set the index to be based off of the GPU_Manufacturer and their respective Id
merged_gpu = merged_gpu.set_index(['GPU_Manufacturer','Id'])

In [253]:
# Drop any duplicates in the dataframe and finally sort by the manufacturer for clear identification
merged_gpu = merged_gpu.drop_duplicates()
merged_gpu = merged_gpu.sort_index()

In [254]:
merged_gpu

Processor_Manufacturer        Processor  \
GPU_Manufacturer Id                                             
AMD              5                       AMD     FireGL V3300   
                 5                       AMD     FireGL V3300   
                 6                       AMD     FireGL V3600   
                 6                       AMD     FireGL V3600   
                 6                       AMD     FireGL V3600   
...                                      ...              ...   
Zotac            1975                 NVidia  GeForce Titan X   
                 1975                 NVidia  GeForce Titan X   
                 1975                 NVidia  GeForce Titan X   
                 1975                 NVidia  GeForce Titan X   
                 1975                 NVidia  GeForce Titan X   

                       Memory_Capacity Memory_Type    Price_USD  \
GPU_Manufacturer Id                                               
AMD              5               0.125       GDDR2    15.860481   
                 5               0.125       GDDR2    16.718548   
                 6               0.250       GDDR2   207.294087   
                 6               0.250       GDDR2   208.495372   
                 6               0.250       GDDR2   206.468852   
...                                ...         ...          ...   
Zotac            1975           12.000       GDDR5  1338.216947   
                 1975           12.000       GDDR5  1080.192556   
                 1975           12.000       GDDR5  1052.097906   
                 1975           12.000       GDDR5  1053.033193   
                 1975           12.000       GDDR5  1034.870641   

                              Country            Merchant  
GPU_Manufacturer Id                                        
AMD              5     United Kingdom    pricespy_unknown  
                 5     United Kingdom    pricespy_unknown  
                 6            Germany    geizhals_unknown  
                 6            Germany    geizhals_unknown  
                 6            Germany    geizhals_unknown  
...                               ...                 ...  
Zotac            1975  United Kingdom  Overclockers.co.uk  
                 1975          Canada               Vuugo  
                 1975          Canada               Vuugo  
                 1975          Canada               Vuugo  
                 1975          Canada               Vuugo  

[913627 rows x 7 columns]

In [255]:
cpu_price_df

,ProdId,TimeId,Price_USD,Price_Original,Country,Merchant
0,1,20160410,30.228604,40.00,Austrailia,IJK
1,1,20160703,30.749616,41.00,Austrailia,IJK
2,1,20170105,30.064161,41.00,Austrailia,IJK
3,2,20160410,37.785755,50.00,Austrailia,IJK
4,2,20160413,39.107430,51.00,Austrailia,IJK
...,...,...,...,...,...,...
1605186,425,20180330,1304.228304,1057.99,Italy,Alternate Italia
1605187,425,20180405,1294.375287,1057.99,Italy,Alternate Italia
1605188,425,20180410,1306.313703,1057.99,Italy,Alternate Italia
1605189,1023,20180316,78.508233,63.89,Italy,Alternate Italia


In [256]:
cpu_product_df

,Id,Manufacturer,Series,CPU_Name,Cores,Socket
0,1,AMD,Amd 2650,Amd Sempron 2650,2,AMD Socket AM1
1,2,AMD,Amd 3850,Amd Sempron 3850,4,AMD Socket AM1
2,3,AMD,Amd 5150,Amd Athlon 5150,4,AMD Socket AM1
3,4,AMD,Amd 5350,Amd Athlon 5350,4,AMD Socket AM1
4,5,AMD,Amd A-Series,Amd A10 Pro-7850B,4,AMD Socket FM2+
...,...,...,...,...,...,...
1659,1660,INTEL,Intel Xeon,Intel Xeon X5680,6,Intel Socket LGA1366
1660,1661,INTEL,Intel Xeon,Intel Xeon X5690,6,Intel Socket LGA1366
1661,1662,INTEL,Intel Xeon,Intel Xeon X6550,8,Intel Socket LGA1567
1662,1663,INTEL,Intel Xeon,Intel Xeon X7350,4,Intel Socket LGA604


In [257]:
# Repeat the same process as for the GPU data frame but set index as just 'Manufacturer'
merged_cpu = pd.merge(cpu_product_df,cpu_price_df,left_on="Id",right_on="ProdId")
merged_cpu = merged_cpu.drop(columns=['ProdId','TimeId','Price_Original'])
merged_cpu = merged_cpu.set_index(['Manufacturer','Id'])
merged_cpu = merged_cpu.drop_duplicates()
merged_cpu = merged_cpu.sort_index()
merged_cpu

Series          CPU_Name  Cores               Socket  \
Manufacturer Id                                                               
AMD          1       Amd 2650  Amd Sempron 2650      2       AMD Socket AM1   
             1       Amd 2650  Amd Sempron 2650      2       AMD Socket AM1   
             1       Amd 2650  Amd Sempron 2650      2       AMD Socket AM1   
             1       Amd 2650  Amd Sempron 2650      2       AMD Socket AM1   
             1       Amd 2650  Amd Sempron 2650      2       AMD Socket AM1   
...                       ...               ...    ...                  ...   
INTEL        1664  Intel Xeon  Intel Xeon X7460      6  Intel Socket LGA604   
             1664  Intel Xeon  Intel Xeon X7460      6  Intel Socket LGA604   
             1664  Intel Xeon  Intel Xeon X7460      6  Intel Socket LGA604   
             1664  Intel Xeon  Intel Xeon X7460      6  Intel Socket LGA604   
             1664  Intel Xeon  Intel Xeon X7460      6  Intel Socket LGA604   

                    Price_USD         Country          Merchant  
Manufacturer Id                                                  
AMD          1      30.228604      Austrailia               IJK  
             1      30.749616      Austrailia               IJK  
             1      30.064161      Austrailia               IJK  
             1      30.901904      Austrailia   Storm Computers  
             1      34.317090      Austrailia   Storm Computers  
...                       ...             ...               ...  
INTEL        1664  323.647829  United Kingdom  pricespy_unknown  
             1664  897.567992  United Kingdom  pricespy_unknown  
             1664  327.581853  United Kingdom  pricespy_unknown  
             1664  324.458187  United Kingdom  pricespy_unknown  
             1664  322.988716  United Kingdom  pricespy_unknown  

[1518379 rows x 7 columns]

In [258]:
ram_price_df

,ProdId,TimeId,Price_USD,Price_Original,Country,Merchant
0,1,20130322,13.749032,10.65,Germany,geizhals_unknown
1,1,20130323,13.828708,10.65,Germany,geizhals_unknown
2,1,20130326,13.694297,10.65,Germany,geizhals_unknown
3,1,20130327,13.690530,10.65,Germany,geizhals_unknown
4,1,20130328,13.605216,10.65,Germany,geizhals_unknown
...,...,...,...,...,...,...
3412326,2446,20180215,227.245761,181.95,Belgium,Bytes At Work
3412327,2446,20180301,217.389976,177.95,Belgium,Bytes At Work
3412328,2446,20180307,214.411901,172.95,Belgium,Bytes At Work
3412329,2446,20180410,216.012989,174.95,Belgium,Bytes At Work


In [259]:
ram_product_df

,Id,Manufacturer,RAM_Name,Memory_Type,Speed,Capacity
0,2,ADATA,Adata,DDR,333,1.0
1,1,ADATA,Adata,DDR,400,0.5
2,3,ADATA,Adata,DDR,400,1.0
3,4,ADATA,Adata,DDR,400,2.0
4,5,ADATA,Adata,DDR2,667,1.0
...,...,...,...,...,...,...
3699,3699,WINTEC,Wintec,DDR3,1866,16.0
3700,3701,WINTEC,Wintec,DDR3,1866,32.0
3701,3703,WINTEC,Wintec,DDR3,1866,64.0
3702,3704,WINTEC,Wintec,DDR4,2133,64.0


In [260]:
# Repeat the same process as for the GPU data frame but set index as just 'Manufacturer'
merged_ram = pd.merge(ram_product_df,ram_price_df,left_on="Id",right_on="ProdId")
merged_ram = merged_ram.drop(columns=['ProdId','TimeId','Price_Original'])
merged_ram = merged_ram.set_index(['Manufacturer','Id'])
merged_ram = merged_ram.drop_duplicates()
merged_ram = merged_ram.sort_index()
merged_ram

RAM_Name Memory_Type  Speed  Capacity   Price_USD  \
Manufacturer Id                                                       
ADATA        1       Adata         DDR    400       0.5   13.749032   
             1       Adata         DDR    400       0.5   13.828708   
             1       Adata         DDR    400       0.5   13.694297   
             1       Adata         DDR    400       0.5   13.690530   
             1       Adata         DDR    400       0.5   13.605216   
...                    ...         ...    ...       ...         ...   
WINTEC       3704   Wintec        DDR4   2133      64.0  442.490000   
             3704   Wintec        DDR4   2133      64.0  441.656667   
             3704   Wintec        DDR4   2133      64.0  433.980000   
             3704   Wintec        DDR4   2133      64.0  642.980000   
             3704   Wintec        DDR4   2133      64.0  614.130000   

                                            Country          Merchant  
Manufacturer Id                                                        
ADATA        1                              Germany  geizhals_unknown  
             1                              Germany  geizhals_unknown  
             1                              Germany  geizhals_unknown  
             1                              Germany  geizhals_unknown  
             1                              Germany  geizhals_unknown  
...                                             ...               ...  
WINTEC       3704  United States Of AMERICA RAAH🦅🦅🦅            Newegg  
             3704  United States Of AMERICA RAAH🦅🦅🦅            Newegg  
             3704  United States Of AMERICA RAAH🦅🦅🦅            Newegg  
             3704  United States Of AMERICA RAAH🦅🦅🦅            Newegg  
             3704  United States Of AMERICA RAAH🦅🦅🦅            Newegg  

[3157280 rows x 7 columns]